In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
import lightkurve as lk
from scipy.optimize import curve_fit
import pandas as pd

In [ ]:
# Specify the Eclipsing CVs: 
TIC = 'TIC 337646214'

data_all = lk.search_lightcurve(TIC, author = 'SPOC')

data_all

In [ ]:
data = data_all[0]

lc = data.download_all().stitch()

In [ ]:
# Define initial guesses for A, mu, and sigma
A_guess = 0.675
mu_guess = 2500.47
sigma_guess = 0.1

# Get the period in julian days
period_julian_days = 0.3007600205719854

# Period of the eclipsing binar
P = 0.3007600205719854

In [ ]:
from decimal import Decimal, getcontext
import numpy as np
from scipy.optimize import curve_fit

# Define the Gaussian function
def gaussian_function(x, A, mu, sigma):
    return -(A * np.exp(-((x - mu)**2) / (2 * sigma**2))-1)

getcontext().prec = 30

def get_O_values(start_index, end_index, mu_fit):
    Times = lc.time[start_index:end_index]
    t = Time(Times)
    x = np.array(t.btjd)
    y = np.array(lc.flux[start_index:end_index])
    
    # Remove invalid values (infs and NaNs) from y and x arrays
    mask_valid = ~np.isnan(y) & ~np.isinf(y)
    x, y = x[mask_valid], y[mask_valid]
    
    if len(x) > 3:

        try:
            params, covariance = curve_fit(gaussian_function, x, y, p0=[A_guess, mu_fit, sigma_guess])
            mu_fit = params[1]  # Update mu_fit with the adjusted value
            
            # Get fitted values for A, μ and σ
            A_fit, mu_fit, sigma_fit = params
            
            # Get the diagonals of the covariance matrix as the variance of μ
            var_mu_fit = np.diag(covariance)[1] 
            
            # Check mu variances and calculate errors
            var_mu_fit_decimal = Decimal(var_mu_fit)
            error_mu_fit = var_mu_fit_decimal.sqrt() 
            error_mu_fit = float(error_mu_fit)
        
            #Add to the lists
            mu_fit_values.append(mu_fit)
            error_mu_fit_values.append(error_mu_fit)
            
        except RuntimeError:
            print("Curve fitting failed to converge")

    else:
        mu_fit = 2514.5  # Valor predeterminado si x es demasiado pequeño

    return mu_fit

# Create the initial_guess list
initial_guess = [A_guess, mu_guess, sigma_guess]

# Initialize mu_fit with the initial guess value for mu
mu_fit = initial_guess[1]

# Initialize lists for mu_fit and error_mu_fit values
mu_fit_values = []
error_mu_fit_values = []

# Iterate through the light curve and process chunks of data with the given period
start_index = 0
i=1
while start_index < len(lc) and i < 59:
    end_time = lc.time[start_index] + period_julian_days
    end_index = np.searchsorted(lc.time, end_time, side='right')
    
    # Call process_chunk with the updated mu_fit value
    mu_fit = get_O_values(start_index, end_index, mu_fit)
    mu_fit = mu_fit + period_julian_days
    
    # Update the start_index for the next iteration
    start_index = end_index
    i=i+1 

#Print O values data table 

df = pd.DataFrame({'O Values': mu_fit_values, 'O Values Errors': error_mu_fit_values})
print(df)


In [ ]:
def get_OC_values(mu_fit_values, P):
    
    # Given observed times (O) of minima for an eclipsing binary
    observed_times = np.array(mu_fit_values)

    T0 = mu_fit_values [0]  # The first mu_fit is T0

    # Calculate the epoch (E) for each mu_fit (O) value
    epochs = np.round((mu_fit_values - T0) / P)

    # Calculate the calculated times (C) using the linear ephemeris model
    calculated_times = T0 + P * epochs

    # Calculate the O-C values
    oc_values = observed_times - calculated_times
    
    return epochs, calculated_times, oc_values

# Create a DataFrame to store the results
epochs, calculated_times, oc_values = get_OC_values(mu_fit_values, P)
oc_data_table = pd.DataFrame({'mu_fit': mu_fit_values,'Epochs': epochs,'Calculated Times': calculated_times,'O-C Values': oc_values})
print(oc_data_table)

In [ ]:
# Fit a linear model to the O-C values vs. epochs
def linear_model(x, a):
    return a * x

params, covariance = curve_fit(linear_model, calculated_times, oc_values)


# The slope of the linear fit represents P-dot (dP/dt)
p_dot = params[0]

# Print the calculated P-dot value
print(f"Calculated P-dot: {p_dot}")

In [ ]:
def graph():
    import numpy as np
    import matplotlib.pyplot as plt
    from numpy.polynomial.polynomial import Polynomial

    #data points
    x = np.array(calculated_times)
    y = np.array(oc_values)

    # Fit a polynomial curve
    degree = 1
    coefs = np.polyfit(x, y, degree)

    # Create a polynomial
    p = Polynomial(coefs[::-1])

    # Print the curve equation
    equation = "y = {:.4f}x + {:.4f}".format(coefs[0], coefs[1])
    print("Best fit equation:")
    print(equation)

    # Graficar
    xp = np.linspace(min(calculated_times), max(calculated_times), 100)
    plt.ylim([-0.05, 0.05])
    plt.scatter(x, y,label='TIC 337646214', marker='.', color='c',)
    #plt.plot(xp, p(xp), '-', label='Fitted curve', color='black')
    plt.xlabel('Calculated Times (BTJD)')
    plt.ylabel('O-C Value')
    plt.title('O-C Diagram')
    plt.legend()
    plt.grid(False)
    plt.show()

graph()
